In [18]:
import requests
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.oauth2 import service_account
from dotenv import load_dotenv
import json
import pprint
from typing import List

load_dotenv()

True

In [6]:
api_config = {
    "scopes": [
        "https://www.googleapis.com/auth/youtube.readonly"
    ],
    "api_service_name": "youtube",
    "api_version": "v3",
    "key_file_path": "key_youtube-stats-459404-eefde03eff46.json"
}

def create_service_account_credentials(api_config: dict) -> service_account.Credentials:
    return service_account.Credentials.from_service_account_file(
        filename=api_config["key_file_path"],
        scopes=api_config["scopes"]
    )

def create_youtube_api_resource(credentials: service_account.Credentials) -> googleapiclient.discovery.Resource:
    return googleapiclient.discovery.build(
        serviceName=api_config["api_service_name"], 
        version=api_config["api_version"], 
        credentials=credentials
    )

In [19]:
def get_channel_info(
        youtube_resource: googleapiclient.discovery.Resource = None, 
        id: str = None
    ) -> dict:

    request = youtube_resource.channels().list(
        part="snippet,contentDetails,statistics,topicDetails,status",
        id=id
    )
    return request.execute()


def parse_channel_uploads_playlist_id(response_channel: dict) -> str:
    return response_channel["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]


def paginate_all_channel_uploads(
        youtube_resource: googleapiclient.discovery.Resource, 
        uploads_playlist_id: str
    ) -> List[dict]:
    def get_uploads_playlist_items(next_page_token: str = None):
        request = youtube_resource.playlistItems().list(
            part="snippet,contentDetails,status",
            maxResults=50,
            playlistId=uploads_playlist_id,
            pageToken=next_page_token
        )
        response_playlist_items = request.execute()

        return response_playlist_items

    video_metadata = []
    response_playlist_items = get_uploads_playlist_items()
    video_metadata.extend(response_playlist_items["items"])

    while response_playlist_items.get("nextPageToken"):
        response_playlist_items = get_uploads_playlist_items(response_playlist_items.get("nextPageToken"))
        video_metadata.extend(response_playlist_items["items"])

    return video_metadata

In [20]:
credentials = create_service_account_credentials(api_config)
youtube_resource = create_youtube_api_resource(credentials)

# identify channel
lirik_plays_channel_id = "UCebh6Np0l-DT9LXHrXbmopg"
response_channel = get_channel_info(youtube_resource, lirik_plays_channel_id)

# identify uploads playlist, pull all videos
uploads_playlist_id = parse_channel_uploads_playlist_id(response_channel)
all_video_metadata = paginate_all_channel_uploads(youtube_resource, uploads_playlist_id)

In [21]:
len(all_video_metadata)

5119